## NEXRAD Training Steps

### Datasets (NEXRAD Level-III Products)

NEXRAD Level-III data is measured at 6 elevation angles, and they are approximately: $0.5^{\circ}$, $0.8^{\circ}$, $1.5^{\circ}$, $1.8^{\circ}$, $2.4^{\circ}$, and $3.4^{\circ}$. Four variables (scan type: PPI), which are Horizontal Reflectivity ($Z_H$), Differential Reflectivity ($Z_{DR}$), Correlation Coefficient ($\rho_{HV}$) and Specific Differential Phase ($K_{DP}$), are used to build input matrix for CNN.

In this project, we use the data from Vance AFB, which is measured at $0.5^{\circ}$ from 2015-01-01 to 2018-10-01. We also downloaded data from 2018-11-01 to 2019-04-30 for future model tests.

The raw datasets can be downloaded from https://www.ncdc.noaa.gov/has/HAS.FileAppRouter?datasetname=7000&subqueryby=STATION&applname=&outdest=FILE (Remember to select **KVNX** as the station). An sample dataset name is like this NWS_NEXRAD_NXL3_KVNX_20181230000000_20181230235959.tar.gz.

### Training Machine

NVIDIA DGX (dgx-dl03.ornl.gov)

### Source Code

https://github.com/YupingLu/Radar

### Step 1: Extract specific files from NEXRAD Level-III data

**nexrad.sh** (https://github.com/YupingLu/Radar/blob/master/script/nexrad.sh) 

Need to specify the following directories in the code:

+ search_dir -> Raw data
+ N0R_dir -> temp directory for variable N0R
+ N0X_dir -> temp directory for variable N0X
+ N0C_dir -> temp directory for variable N0C
+ N0K_dir -> temp directory for variable N0K
+ N0H_dir -> temp directory for variable N0H
+ target_dir -> Final data for next step

The code also generates lists of filenames of each variables, and they are n0r.txt n0x.txt n0c.txt n0k.txt and n0h.txt.

### Step 2: Crop raw data to create $12*6$ $30*30$ matrices

**datacrop3.py** (https://github.com/YupingLu/Radar/blob/master/script/datacrop3.py) 

Need to specify the following directories in the code:

- target_dir and lists of filenames in the step 1.
- name2 and data2 to store temp files.

The code also genrate a summary about the number of each category. In this project, we only use four categories and they are 30 Ice Crystals (IC), 40 Dry Snow (DS), 60 Light and/or Moderate Rain (RA), and 80 Big Drops (rain) (BD).

### Step 3: Extract datasets from datacrop3

**extract.py** (https://github.com/YupingLu/Radar/blob/master/script/extract.py)

First, specify the number of datasets to extract for each category. Use the results from step 2. Here is an example:

cnt = {
    '30': 10192,
    '40': 25974,
    '60': 19166,
    '80': 19166,
}

Need to specify the following directories in the code:

- name2 and data2 in step 2.
- data3 to store temp fiels.
- also four sub directories in data3, and they are 30, 40, 60, and 80.

### Step 4: Combine four variable datasets into one

**combine.py** (https://github.com/YupingLu/Radar/blob/master/script/combine.py)

Need to specify the following directories in the code:

- data3 in step 3.
- dataloader to store final input files.
- also four sub directories in dataloader, and they are 30, 40, 60, and 80.

### Step 5: Split datasets into training, validation and test set

**split2.py** (https://github.com/YupingLu/Radar/blob/master/script/split2.py)

Remember to make necessary changes in the script.

After running this script, change the names of four sub directories in dataloader's train directory, validation directory and test directory:

- 30 -> IC
- 40 -> DS
- 60 -> RA
- 80 -> BD

### Step 5: Training setup

Create a directory to do the training. For example, my directory is /home/ylk/workspace/t-08042019-01.

Copy file main.py and directory datasets and models from github repo to this directory.

Create an empty directory checkpoint to store trained models.

Use **nvidia-smi** to check available GPUs on dgx. Take the example below, GPU 0 and 3 are available.

    (base) ylk@dgx-dl03:/raid/ylk/dataloader$ nvidia-smi
    Tue Aug  6 14:16:56 2019
    +-----------------------------------------------------------------------------+
    | NVIDIA-SMI 384.183      Driver Version: 384.183      CUDA Version: 9.0      |
    |-------------------------------+----------------------+----------------------+
    | GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
    | Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
    |===============================+======================+======================|
    |   0  Tesla V100-DGXS...  On   | 00000000:07:00.0 Off |                    0 |
    | N/A   43C    P0    40W / 300W |     28MiB / 16149MiB |      0%      Default |
    +-------------------------------+----------------------+----------------------+
    |   1  Tesla V100-DGXS...  On   | 00000000:08:00.0 Off |                    0 |
    | N/A   44C    P0    54W / 300W |   1524MiB / 16149MiB |     12%      Default |
    +-------------------------------+----------------------+----------------------+
    |   2  Tesla V100-DGXS...  On   | 00000000:0E:00.0 Off |                    0 |
    | N/A   46C    P0   140W / 300W |   3694MiB / 16149MiB |     44%      Default |
    +-------------------------------+----------------------+----------------------+
    |   3  Tesla V100-DGXS...  On   | 00000000:0F:00.0 Off |                    0 |
    | N/A   43C    P0    40W / 300W |     10MiB / 16149MiB |      0%      Default |
    +-------------------------------+----------------------+----------------------+
    
    +-----------------------------------------------------------------------------+
    | Processes:                                                       GPU Memory |
    |  GPU       PID   Type   Process name                             Usage      |
    |=============================================================================|
    |    0      1570      G   /usr/lib/xorg/Xorg                            17MiB |
    |    1       397      C   python                                      1514MiB |
    |    2       433      C   python                                      3684MiB |
    +-----------------------------------------------------------------------------+

### Step 6: Training

Specify the path of training dataset and validation dataset in main.py. For example:

    trainset = NexradDataset(root='/raid/ylk/dataloader/train/', transform=train_transform)
    validationset = NexradDataset(root='/raid/ylk/dataloader/validation/', transform=validation_transform)

An exmaple:

    nohup python main.py --arch resnet18 --gpu-id 1 > resnet18-08042019-01.out 2> resnet18-08042019-01.err < /dev/null &

Here the hyper-parameters are: RandomCrop(), RandomHorizontalFlip(), RandomVerticalFlip(), batch-size = 256, epochs = 600, lr = 0.1, momentum = 0.9, weight-decay = 1e-3, dlr-decay = 0.5.